##### Import all of the necessary packages.

In [3]:
import random
import os
import sys
import datetime
from enum import Enum
from typing import Optional
from abc import ABC, abstractmethod
from IPython.display import clear_output

##### We'll take a modular approach by defining the different parts of the games as classes to be instanced.

Below, cls refers to the class itself and self refers to the specific instance of the class (i.e. whatever choice it is).

In [17]:
class GameChoice(Enum):
    """Simple Enums representing different game choices."""
    # Leave open to add more valid inputs
    ROCK = 'r'
    PAPER = 'p'
    SCISSORS = 's'

    @classmethod
    def from_input(cls, user_input: str) -> Optional['GameChoice']:
        """Go through different choices in class and return the one that matches the user input."""
        for choice in cls:
            if user_input.lower() == choice.value:
                return choice
        return None

    def __str__(self) -> str:
        """Return the string representation of the game choice."""
        return self.name.capitalize()


In [5]:
class GameResult(Enum):
    """The player results of each game won in the current session."""
    P1_WIN = 1
    P2_WIN = 2
    TIE = 0

##### Define an abstract base class for the human and computer players.

Since there will be multiple types of "players" we can use python's abstract class object as an interface to define common characteristics of each player. The abstractmethod decorator indicates that the function must be implemented by any child class.

In [6]:
class AbstractPlayer(ABC):
    """Abstract base class used for all player-type objects."""

    def __init__(self, name: str):
        """
        Initialize a player object with a name.

        :param name: Player's name.
        """

        self.name = name
        self.score = 0
        self.won_rounds = []

    @abstractmethod
    def get_choice(self) -> GameChoice:
        """Get the player's input/choice for the round."""
        pass  # Will define this later for the child classes

    def reset_score(self):
        """Resets the given player instance's score and rounds they have won."""
        self.score = 0
        self.won_rounds = []

    def add_round_win(self, round_number:int):
        self.score += 1
        self.won_rounds.append(round_number)

    def __str__(self):
        """Return the string representation of the player."""
        return f"{self.name}: {self.score}"



##### Define the inherited player classes with their own individual methods and attributes.

super() is a built-in function that returns that gives us a way to access the parent class's methods and attributes. The decorator @staticmethod only indicates to python that the function doesn't return anything.

The computer player class only decides via using random.choice.

In [7]:
class HumanPlayer(AbstractPlayer):
    """Implementation of human player from abstract player."""

    def __init__(self, name: str, hide_input: bool = False):
        """
        Initialize a human player using abstract player __init__ method.

        Args:
            name: The player's name
            hide_input: Whether to clear screen after input (for PvP mode)
        """
        super().__init__(name)
        self.hide_input = hide_input

    def get_choice(self) -> GameChoice:
        """
        Get the human player's choice with input validation

        Returns:
            The player's choice as an Enum derived from GameChoice
        """
        while True:
            user_input = input(f"{self.name}, enter your choice (r/p/s): ").lower()
            user_choice = GameChoice.from_input(user_input)

            if user_choice:
                if self.hide_input:
                    self._clear_screen()
                return user_choice
            else:
                print("Invalid input! Please enter 'r' for rock, 'p' for paper, or 's' for scissors.")

    # Private function that should not be accessed outside of class and don't want to pass self
    @staticmethod
    def _clear_screen():
        """Clear any text in the console screen."""
        # os.system('clear' if os.name == 'posix' else 'cls')
        # clear_output(wait=True)
        # Gets messy with notebook...

class ComputerPlayer(AbstractPlayer):
    """Implementation of computer player from abstract player."""

    def __init__(self, name: str = "Computer"):
        """
        Initialize a computer player using abstract player __init__ method.

        Args:
            name: The player's name
        """
        super().__init__(name)

    def get_choice(self) -> GameChoice:
        """
        Get the computer player's choice and validate its input.

        Returns:
            The player's choice as an Enum derived from GameChoice
        """
        return random.choice(list(GameChoice))  # We can parse our Enums from GameChoice and select randomly.

##### Next we need to handle the game logic behind how each round is played.

Overall this is pretty simple, the main interesting part is using the previous classes' functions and attributes.

In [34]:
class HandleRound:  # Do not need to inherit from any python ABC
    """Controls all game logic for each round."""

    @staticmethod
    def determine_round_winner(choice1: GameChoice, choice2: GameChoice) -> GameResult:
        """
        Determine the winner of a round based on each of their individual choices.

        :param choice1: Player 1 choice
        :param choice2: Player 2 choice
        :return: Updated GameResult Enum
        """

        #  Define what the wining conditions look like for one player
        winning_conditions = dict(
            Condition_One = (GameChoice.ROCK, GameChoice.SCISSORS),
            Condition_Two = (GameChoice.PAPER, GameChoice.ROCK),
            Condition_Three = (GameChoice.SCISSORS, GameChoice.PAPER)
        )
        
        if choice1 == choice2:
            return GameResult.TIE
        elif (choice1, choice2) in winning_conditions.values():
            return GameResult.P1_WIN
        else:
            return GameResult.P2_WIN

    @staticmethod
    def play_round(player1: AbstractPlayer, player2: AbstractPlayer, round_number:int, max_rounds:int) -> GameResult:
        """
        Start to play a new round between two players.

        :param player1: First player
        :param player2: Second player
        :param round_number: Current round
        :return: Result of the current round
        """

        print(f"\n---- ROUND {round_number} OF {max_rounds} ----")

        #  Access the choices from both player classes (could be computer or human)
        choice1 = player1.get_choice()
        choice2 = player2.get_choice()

        #  Print the choices of each player
        print(f"\n{player1.name} has chosen: {choice1} and")
        print(f"{player2.name} has chosen: {choice2}")

        # Determine the winner and loser of the round
        result = HandleRound.determine_round_winner(choice1, choice2)

        #  Print the result of the round and update the rounds won for each player
        if result == GameResult.P1_WIN:
            player1.add_round_win(round_number)
            print(f"\n{player1.name} wins this round!")
        elif result == GameResult.P2_WIN:
            player2.add_round_win(round_number)
            print(f"\n{player2.name} wins this round!")
        else:
            print("\nTie! No points will be awarded to either player.")

        return result


##### Next we need to define the main game loop which will handle all of our logic per-game (not per-round!).

There are two main classes which handle the session (which is the actual instance of our game in a class: RockPaperScissorsGame): the HandleRound class and the HandleGame class. Each game consists of a maximum of three rounds; all of the logic behind this is defined below.

In [38]:
class HandleGame:
    """Manages a complete game loop and related logic."""

    def __init__(self, player1: AbstractPlayer, player2: AbstractPlayer, console: ConsoleMenu, max_rounds: int = 3):
        """
        Initializes a game with default of 3 rounds.

        :param player1: First player
        :param player2: Second player
        :param max_rounds: Max number of rounds to play
        """

        self.player1 = player1
        self.player2 = player2
        self.console = console
        self.max_rounds = max_rounds
        self.round_handler = HandleRound()
        self.current_round = 0
        self.is_game_active = True

    def check_wins(self, player: AbstractPlayer) -> bool:
        """
        Check if either player has won consecutively twice.
        
        :param player: Player to check
        :return: True if player has consecutive wins
        """
        if len(player.won_rounds) < 2:
            return False
        else:
            sorted_rounds = sorted(player.won_rounds)
            for i in range(len(sorted_rounds)-1):
                if sorted_rounds[i+1] - sorted_rounds[i] != 1:
                    return True
        return False

    def display_current_score(self):
        """Print the current game score."""
        print(f"\nCURRENT SCORE:")
        print(f"{self.player1.name} has: {self.player1.score} wins\n")
        print(f"{self.player2.name} has: {self.player2.score} wins")

    def _check_continue(self) -> bool:
        """
        Checks if the player want to quit the game.
        :return: True or false depending on player choice.
        """
        while True:
            choice_to_continue = input("\nContinue? (y/n) for Yes/No or (q) to Quit to menu: ").lower()
            if choice_to_continue in ['y', 'yes']:
                return True
            elif choice_to_continue in ['n', 'q', 'no', 'quit']:
                self.is_game_active = False
                return False
            else:
                print("Invalid input. Please enter 'y' to continue or 'n'/'q' to quit.")

    def play_game(self) -> Optional[AbstractPlayer]:
        """
        Initiate a completely new game session.

        :return: The winning player if there is one.
        """
        # Determine how many rounds where other player/computer cannot win
        if (self.max_rounds % 2) == 0:
            majority_win_num = self.max_rounds + 1
        else:
            majority_win_num = round(self.max_rounds/2)
            
        print(f"\n{'=' * 40}")
        print(f"Starting new game: {self.player1.name} vs. {self.player2.name}")
        print(f"First to win {majority_win_num} out of {self.max_rounds} rounds totally or 2 consecutively wins the game!")
        print(f"{'=' * 40}")

        # Reset player scores if they are not already.
        self.player1.reset_score()
        self.player2.reset_score()

        for round_num in range(1, self.max_rounds + 1):
            if not self.is_game_active:
                break

            self.current_round = round_num

            # Check for quit option
            if not self._check_continue():
                return None

            # Play the round
            self.round_handler.play_round(self.player1, self.player2, round_num, self.max_rounds)

            # Display the current score
            self.display_current_score()

            # Check for consecutive wins (automatic win condition) - Optional
            if self.check_wins(self.player1):
                self.console.print_game_over()
                print(f"\n{self.player1.name} wins the game after 2 consecutive rounds!")
                return self.player1
            elif self.check_wins(self.player2):
                self.console.print_game_over()
                print(f"\n{self.player2.name} wins the game after 2 consecutive rounds!")
                return self.player2

            # Check if someone has already won most of the rounds
            if self.player1.score >= majority_win_num:
                self.console.print_game_over()
                print(f"\n{self.player1.name} wins the game!")
                return self.player1
            elif self.player2.score >= majority_win_num:
                self.console.print_game_over()
                print(f"\n{self.player2.name} wins the game!")
                return self.player2

            # Add a pause between rounds (except after the last round)
            if round_num < self.max_rounds:
                input("\nPress Enter to continue to the next round...")

        # If we've played all rounds and no one has won consecutively or by majority; check scores
        self.console.print_game_over()
        if self.player1.score > self.player2.score:
            print(f"\n{self.player1.name} wins the game!")
            return self.player1
        elif self.player2.score > self.player1.score:
            print(f"\n{self.player2.name} wins the game!")
            return self.player2
        else:
            print("\nThe game ends in a tie!")
            return None

##### Create a menu in the console that player(s) can interact with.

There are many other options in python to define a menu, but for simplicty the console is the easiest. Using the console, we can just print all of the information through text via python printing tricks.

In [41]:
class ConsoleMenu:
    """Handles the logic behind the game menu and navigation in the console."""
    
    def __init__(self):
        """Initialize the console menu."""
        self.running = True

    @staticmethod #  Class methods will automatically try to pass self unless decorator is used
    def display_console_menu():
        """Display the options in the console menu."""
        print("\n" + "=" * 40)
        print("GAME MODES")
        print("=" * 40)
        print("1. Player vs. Player")
        print("2. Player vs. Computer")
        print("3. Quit")
        print("=" * 40)

    @staticmethod
    def print_game_over():
        print("\n" + "=" * 40 + "\nGAME OVER\n" + "=" * 40)

    @staticmethod
    def print_ascii_copypasta():
        ascii_intro_text = open("ascii_intro.txt", 'r')
        for line in ascii_intro_text.readlines():
            print(line.rstrip())

    @staticmethod
    def get_menu_choice() -> str:
        """
        Get the user's choice from the console and validate it.

        :return: Selected menu option as a string.
        """
        while True:
            menu_choice = input("Enter your choice (1-3): ")
            if menu_choice in ['1', '2', '3']:
                return menu_choice
            else:
                print("Invalid choice. Please enter 1, 2, or 3.")
                
    @staticmethod
    def get_player_name(player_number: int) -> str:
        """
        Get a player's name based on their number.

        :param player_number: Player number to print
        :return: Player chosen name as string
        """

        # .strip() gets rid of any white space
        name = input(f"Enter Player {player_number}'s name: ").strip()
        # If there is no name return number as name
        return name if name else f"Player {player_number}"


##### Here we define the object which controls the entire game from start to finish.

So, all this means is that we call (instance) all of the previous class objects we defined before. The structure of HOW the game loop works is determined here (i.e. instead of in the class which only handles games or rounds). 

In [50]:
class RockPaperScissorsGame:
    """Main game controller which orchestrates the entire game loop. """

    def __init__(self):
        """Initialize the main game controller."""
        self.menu = ConsoleMenu()
        self.current_game_session = None  # No current instance of HandleGame

    def run(self):
        """The core of the main game loop."""

        print("WELCOME TO: ")
        self.menu.print_ascii_copypasta()
        print(f"\nCURRENT TIME: {datetime.datetime.now()}")

        while self.menu.running:
            self.menu.display_console_menu()
            game_mode_choice = self.menu.get_menu_choice()

            if game_mode_choice == '1':
                self.play_pvp()
            elif game_mode_choice == '2':
                self.play_pvc()
            elif game_mode_choice == '3':
                self.quit_game()

    def play_pvp(self):
        """Run the player vs. player mode."""
        print("\n--- Player vs Player Mode ---")

        # Get player names
        name1 = self.menu.get_player_name(1)
        name2 = self.menu.get_player_name(2)

        # Create player controllers and clear the screen (console) after their turn
        player1 = HumanPlayer(name1, hide_input=True)
        player2 = HumanPlayer(name2, hide_input=True)

        # Create and start the game session
        game_session = HandleGame(player1, player2, self.menu)
        winner = game_session.play_game()

        if winner:
            print(f"\nCongratulations, {winner.name}!")

        self.post_game_options()

    def play_pvc(self):
        """Handle the Player vs. Computer mode."""
        print("\n--- Player vs Computer Mode ---")

        # Get player name
        name = self.menu.get_player_name(1)

        # Create player and computer controllers
        player1 = HumanPlayer(name, hide_input=False)
        player2 = ComputerPlayer()

        # Create and start the game session
        game_session = HandleGame(player1, player2, self.menu)
        winner = game_session.play_game()

        if winner:
            print(f"\nGame Over! {winner.name} wins!")

        self.post_game_options()

    def post_game_options(self):
        """Handles the post-game options."""
        while True:
            post_game_choice = input("\nPlay again? (y/n): ").lower()
            if post_game_choice in ['y', 'yes']:
                return  # Return to main menu
            elif post_game_choice in ['n', 'no']:
                self.quit_game()
                break
            else:
                print("Invalid input. Please enter 'y' or 'n'.")

    def quit_game(self):
        """Quit the game."""
        print(f"\n{'=' * 130}\nTHANK YOU FOR PLAYING: \n")
        self.menu.print_ascii_copypasta()
        print("Goodbye!")
        self.menu.running = False
        sys.exit(0)

##### Finally, let's actually run the program and see how it works!

In [51]:
game_instance = RockPaperScissorsGame()
game_instance.run()

WELCOME TO: 
  _____            _        _____                         _____      _                        _
 |  __ \          | |      |  __ \                       / ____|    (_)                      | |
 | |__) |___   ___| | __   | |__) |_ _ _ __   ___ _ __  | (___   ___ _ ___ ___  ___  _ __ ___| |
 |  _  // _ \ / __| |/ /   |  ___/ _` | '_ \ / _ \ '__|  \___ \ / __| / __/ __|/ _ \| '__/ __| |
 | | \ \ (_) | (__|   < _  | |  | (_| | |_) |  __/ |_    ____) | (__| \__ \__ \ (_) | |  \__ \_|
 |_|  \_\___/ \___|_|\_( ) |_|   \__,_| .__/ \___|_( )  |_____/ \___|_|___/___/\___/|_|  |___(_)
                       |/             | |          |/
                                      |_|

CURRENT TIME: 2025-11-14 18:17:32.540947

GAME MODES
1. Player vs. Player
2. Player vs. Computer
3. Quit


Enter your choice (1-3):  1



--- Player vs Player Mode ---


Enter Player 1's name:  Dan
Enter Player 2's name:  Ben



Starting new game: Dan vs. Ben
First to win 2 out of 3 rounds totally or 2 consecutively wins the game!


...

Dan, enter your choice (r/p/s):  r
Ben, enter your choice (r/p/s):  p



Dan has chosen: Rock and
Ben has chosen: Paper

Ben wins this round!

CURRENT SCORE:
Dan has: 0 wins

Ben has: 2 wins

GAME OVER

Ben wins the game after 2 consecutive rounds!

Congratulations, Ben!



Play again? (y/n):  n



THANK YOU FOR PLAYING: 

  _____            _        _____                         _____      _                        _
 |  __ \          | |      |  __ \                       / ____|    (_)                      | |
 | |__) |___   ___| | __   | |__) |_ _ _ __   ___ _ __  | (___   ___ _ ___ ___  ___  _ __ ___| |
 |  _  // _ \ / __| |/ /   |  ___/ _` | '_ \ / _ \ '__|  \___ \ / __| / __/ __|/ _ \| '__/ __| |
 | | \ \ (_) | (__|   < _  | |  | (_| | |_) |  __/ |_    ____) | (__| \__ \__ \ (_) | |  \__ \_|
 |_|  \_\___/ \___|_|\_( ) |_|   \__,_| .__/ \___|_( )  |_____/ \___|_|___/___/\___/|_|  |___(_)
                       |/             | |          |/
                                      |_|
Goodbye!


SystemExit: 0

##### It seems like everything is working well, let's also try the Player vs. Computer mode.

In [52]:
game_instance = RockPaperScissorsGame()
game_instance.run()

WELCOME TO: 
  _____            _        _____                         _____      _                        _
 |  __ \          | |      |  __ \                       / ____|    (_)                      | |
 | |__) |___   ___| | __   | |__) |_ _ _ __   ___ _ __  | (___   ___ _ ___ ___  ___  _ __ ___| |
 |  _  // _ \ / __| |/ /   |  ___/ _` | '_ \ / _ \ '__|  \___ \ / __| / __/ __|/ _ \| '__/ __| |
 | | \ \ (_) | (__|   < _  | |  | (_| | |_) |  __/ |_    ____) | (__| \__ \__ \ (_) | |  \__ \_|
 |_|  \_\___/ \___|_|\_( ) |_|   \__,_| .__/ \___|_( )  |_____/ \___|_|___/___/\___/|_|  |___(_)
                       |/             | |          |/
                                      |_|

CURRENT TIME: 2025-11-14 18:19:42.491387

GAME MODES
1. Player vs. Player
2. Player vs. Computer
3. Quit


Enter your choice (1-3):  2



--- Player vs Computer Mode ---


Enter Player 1's name:  Daniel



Starting new game: Daniel vs. Computer
First to win 2 out of 3 rounds totally or 2 consecutively wins the game!


...


Play again? (y/n):  y



GAME MODES
1. Player vs. Player
2. Player vs. Computer
3. Quit


Enter your choice (1-3):  3



THANK YOU FOR PLAYING: 

  _____            _        _____                         _____      _                        _
 |  __ \          | |      |  __ \                       / ____|    (_)                      | |
 | |__) |___   ___| | __   | |__) |_ _ _ __   ___ _ __  | (___   ___ _ ___ ___  ___  _ __ ___| |
 |  _  // _ \ / __| |/ /   |  ___/ _` | '_ \ / _ \ '__|  \___ \ / __| / __/ __|/ _ \| '__/ __| |
 | | \ \ (_) | (__|   < _  | |  | (_| | |_) |  __/ |_    ____) | (__| \__ \__ \ (_) | |  \__ \_|
 |_|  \_\___/ \___|_|\_( ) |_|   \__,_| .__/ \___|_( )  |_____/ \___|_|___/___/\___/|_|  |___(_)
                       |/             | |          |/
                                      |_|
Goodbye!


SystemExit: 0

#### It looks like everything works. Overall, I would say the modular class-based approach worked well for this case. In the future, I would try to find ways to optimize the code and account for more edge cases (e.g. notebook environment).